# **Reto II**

### 1. Dataset

Los datos de origen son proporcionados en un archivos csv:

* udfs: dataset con datos de operaciones financieras.

### 2. Columnas y significado:

* nb: número de referencia de la operación.
* contract: identificador de contrato.
* udf_ref: identificador de operación de trading.
* fmly: familia a la que pertenece la operación financiera.
* grp: grupo al que pertenece la operación financiera.
* type: tipo de operación financiera.
* country: país de origen de la operación.
* udf_name: campo informado en el registro.
* num_value: valor numérico.
* string_value: valor de cadena de caracteres.
* date_value: valor de fecha.
* data_timestamp_part: marca temporal.
* data_date_part: fecha en la que se almacena la información.
* source_system: fuente de los datos.

### 3. Descripción del problema:

Si hacemos una visión general a nuestro conjunto de datos, podemos observar como hay hasta 10 registros (filas) para cada valor de *nb*, donde cada registro solo da información para un valor de *udf_name*. Esto es un gasto innecesario de almacenamiento y computación, además de complicar los futuros cálculos derivados de estos datos. Por esta razón, necesitamos convertir estos registros con el mismo *nb* a un solo registro.

Nuestro dataframe final tendrá que contener las siguientes columnas: `nb, M_CCY, M_CLIENT, M_CRDTCHRG, M_DIRECTIAV, M_DISCMARGIN, M_LIQDTYCHRG, M_MVA, M_RVA, M_SELLER, M_SUCURSAL`

* nb: debe contener el número de referencia de la operación.
* M_CLIENT, M_SELLER, M_CCY, M_SUCURSAL: deben mapear el valor de *string_value*
* M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA: deben mapear el valor de *num_value*


Una vez tengamos este resultado, necesitaremos eliminar las operaciones que no tengan informados ninguno de los siguientes campos:

M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, M_MVA, M_RVA, M_SELLER

No informados en este caso significa que o son valores nulos, vacíos o 0, en el caso de los campos numéricos.

### 4. Reto:

* Obtener un dataframe final que contenga las columnas indicadas, con un registro por *nb* y con los valores correctos mapeados.
* Las operaciones con los campos M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA, M_SELLER no informados no deben existir.
* Hacerlo de la manera más eficiente posible a nivel computacional.

**NOTA:** Cada uno de los pasos descritos en el problema pueden efectuarse en una sola línea.

### Inicialización de SparkSession:

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Reto 2")
                        .master("local")
                        .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://L2111027.bosonit.local:4040
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1641556471322)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@2da29696


In [2]:
val udfs = spark.read.format("csv")
                     .option("header", "true")
                     .option("delimiter", ";")
                     .load("./reto2/udfs.csv")

udfs: org.apache.spark.sql.DataFrame = [nb: string, contract: string ... 12 more fields]


### Resultado:

**INSTRUCCIONES**: El DataFrame resultante debe almacenarse en la variable `resultado`, sustituyendo el valor `None` por el código que consideréis oportuno. De esta forma podréis comprobar si el resultado es correcto.

In [3]:
// Husmeando un poco el dataset inicial
udfs.where($"nb"==="2222222").show()

+-------+--------+-------+----+---+----+-------+------------+---------------+------------+----------+-------------------+--------------+-------------+
|     nb|contract|udf_ref|fmly|grp|type|country|    udf_name|      num_value|string_value|date_value|data_timestamp_part|data_date_part|source_system|
+-------+--------+-------+----+---+----+-------+------------+---------------+------------+----------+-------------------+--------------+-------------+
|2222222| 2222222|2222222| IRD|IRS|null|    ESP|  M_CRDTCHRG|30.000000000000|        NULL|      NULL|     20210203032054|    2020-12-30|        Mx3EU|
|2222222| 2222222|2222222| IRD|IRS|null|    ESP|    M_SELLER|           NULL|  LB_TLECLER|      NULL|     20210203032054|    2020-12-30|        Mx3EU|
|2222222| 2222222|2222222| IRD|IRS|null|    ESP|M_LIQDTYCHRG|50.000000000000|        NULL|      NULL|     20210203032054|    2020-12-30|        Mx3EU|
|2222222| 2222222|2222222| IRD|IRS|null|    ESP|       M_MVA|             20|        NULL|    

In [4]:
// Churreteando un poco el dataset inicial
udfs.where($"nb"==="3815982").show()

+-------+--------+-------+----+---+----+-------+------------+--------------+------------+----------+-------------------+--------------+-------------+
|     nb|contract|udf_ref|fmly|grp|type|country|    udf_name|     num_value|string_value|date_value|data_timestamp_part|data_date_part|source_system|
+-------+--------+-------+----+---+----+-------+------------+--------------+------------+----------+-------------------+--------------+-------------+
|3815982| 3672136|4951879| SCF|SCF| SCF|    ESP|M_DISCMARGIN|0.000000000000|        NULL|      NULL|     20200406103021|    2020-12-30|        Mx3EU|
|3815982| 3672136|4951879| SCF|SCF| SCF|    ESP|  M_SUCURSAL|          NULL|        null|      NULL|     20200406103021|    2020-12-30|        Mx3EU|
|3815982| 3672136|4951879| SCF|SCF| SCF|    ESP|       M_RVA|0.000000000000|        NULL|      NULL|     20200406103021|    2020-12-30|        Mx3EU|
|3815982| 3672136|4951879| SCF|SCF| SCF|    ESP|M_LIQDTYCHRG|0.000000000000|        NULL|      NULL|

In [5]:
val pivote1 = udfs.groupBy($"nb").pivot($"udf_name").agg(first($"string_value"))
val pivote2 = udfs.groupBy($"nb").pivot($"udf_name").agg(first($"num_value"))

val joined_pivotes = pivote1.as("a").join(pivote2.as("b"),$"a.nb" === $"b.nb" ,"inner")

val joined_pivotes_limpio = joined_pivotes.select($"a.nb", $"a.M_CCY", $"a.M_CLIENT", $"b.M_CRDTCHRG", $"b.M_DIRECTIAV", $"b.M_DISCMARGIN", $"b.M_LIQDTYCHRG", 
$"b.M_MVA", $"b.M_RVA", $"a.M_SELLER", $"a.M_SUCURSAL")

joined_pivotes_limpio.show(5)

+---------+-----+--------+---------------+--------------+------------+---------------+-----+--------------+----------+----------+
|       nb|M_CCY|M_CLIENT|     M_CRDTCHRG|   M_DIRECTIAV|M_DISCMARGIN|   M_LIQDTYCHRG|M_MVA|         M_RVA|  M_SELLER|M_SUCURSAL|
+---------+-----+--------+---------------+--------------+------------+---------------+-----+--------------+----------+----------+
| 20513130| null|    CCMO| 0.000000000000|0.000000000000|          10| 0.000000000000|   20|0.000000000000|      WATT|      5493|
| 18710605|  MXN|    null|           null|0.000000000000|        null|           null| null|          null|      AMAM|      null|
| 20533916| null|    CCMO| 0.000000000000|0.000000000000|          50| 0.000000000000|   30|0.000000000000|      WATT|      1999|
| 23097010|  EUR|    null|           null|0.000000000000|        null|           null| null|          null|      AMAM|      null|
|555111222|  USD|    null|30.000000000000|0.000000000000|          10|50.000000000000|   2

pivote1: org.apache.spark.sql.DataFrame = [nb: string, M_CCY: string ... 10 more fields]
pivote2: org.apache.spark.sql.DataFrame = [nb: string, M_CCY: string ... 10 more fields]
joined_pivotes: org.apache.spark.sql.DataFrame = [nb: string, M_CCY: string ... 22 more fields]
joined_pivotes_limpio: org.apache.spark.sql.DataFrame = [nb: string, M_CCY: string ... 9 more fields]


In [6]:
val resultado = joined_pivotes_limpio.where(($"M_CCY" !== "NULL") || ($"M_CLIENT" !== "NULL") || (($"M_CRDTCHRG" !== "0.000000000000") && ($"M_CRDTCHRG" !== "NULL")) || (($"M_DIRECTIAV" !== "0.000000000000") && ($"M_DIRECTIAV" !== "NULL")) || (($"M_DISCMARGIN" !== "0.000000000000") && ($"M_DISCMARGIN" !== "NULL")) || (($"M_LIQDTYCHRG" !== "0.000000000000") && ($"M_LIQDTYCHRG" !== "NULL")) || (($"M_RVA" !== "0.000000000000") && ($"M_RVA" !== "NULL")) || (($"M_MVA" !== "0.000000000000") && ($"M_MVA" !== "NULL")) || ($"M_SELLER" !== "NULL") || ($"M_SUCURSAL" !== "NULL"))

resultado.show()

+---------+-----+--------+---------------+-----------------+--------------+---------------+--------------+--------------+----------+----------+
|       nb|M_CCY|M_CLIENT|     M_CRDTCHRG|      M_DIRECTIAV|  M_DISCMARGIN|   M_LIQDTYCHRG|         M_MVA|         M_RVA|  M_SELLER|M_SUCURSAL|
+---------+-----+--------+---------------+-----------------+--------------+---------------+--------------+--------------+----------+----------+
| 20513130| null|    CCMO| 0.000000000000|   0.000000000000|            10| 0.000000000000|            20|0.000000000000|      WATT|      5493|
| 18710605|  MXN|    null|           null|   0.000000000000|          null|           null|          null|          null|      AMAM|      null|
| 20533916| null|    CCMO| 0.000000000000|   0.000000000000|            50| 0.000000000000|            30|0.000000000000|      WATT|      1999|
| 23097010|  EUR|    null|           null|   0.000000000000|          null|           null|          null|          null|      AMAM|    

resultado: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [nb: string, M_CCY: string ... 9 more fields]


In [7]:
assert(resultado.count() == 60)
assert(resultado.columns.size == 11)
assert(resultado.columns(4) == "M_DIRECTIAV")
assert(resultado.select("M_SELLER").filter($"nb" === 23037162).first.getString(0) == "AMAM")
assert(resultado.select("M_SELLER").filter($"nb" === 19665186).first.getString(0) == "LB_VSTAVRE")
assert(resultado.select("M_RVA").filter($"nb" === 444111222).first.getString(0) == "8956")